In [1]:
import time
import pyspark
import datetime
import numpy as np
from functools import reduce
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

##### Utility Function

In [19]:
def get_data(sessn, query):
    """
    Returns the pySpark DataFrame
    
    Parameters:
        sessn : Pyspark Session object
        query (str): The input HQL query
        
    Returns:
        DataFrame (pySpark)
    """
    try:
        return sessn.sql(query)
    except Exception as e:
        print(e)

##### Initiating Pyspark Session

In [2]:
try:
    sc.stop()
except Exception as e:
    print(e)
try:
    sessn.stop()
except Exception as e:
    print(e)
try:
    sc.SparkContext.stop()
except Exception as e:
    print(e)

name 'sessn' is not defined
'SparkContext' object has no attribute 'SparkContext'


In [22]:
conf = pyspark.SparkConf().setAll([('spark.executor.memory', '4g'), 
                                   ('spark.executor.instances', '25'),
                                   ('spark.executor.cores', '4'), 
                                   ('spark.driver.cores','4'), 
                                   ('spark.driver.memory','4g'),
                                   ('spark.yarn.queue', 'araadh_q1.arapi_sq1'),
                                   ('spark.yarn.executor.memoryOverhead', '2g')])
sessn = SparkSession.builder.appName("PS").config(conf=conf).enableHiveSupport().getOrCreate().newSession()
sessn

##### Configuration

In [23]:
# Date Range
lower_dt = "20190701"
upper_dt = "20200930"
db = "pi_usrs"
raw_table = "ps_837p_historical"
acquisition_table  = "{}.{}".format(db, raw_table)
batch_ = 6

In [24]:
start = datetime.datetime.strptime(lower_dt, "%Y%m%d")
end   = datetime.datetime.strptime(upper_dt, "%Y%m%d")
date_list = [(start + datetime.timedelta(days=x)).strftime("%Y%m%d") for x in range(0, (end-start).days+1)]
date_split = {}
date_batch = np.array_split(date_list, batch_)

In [25]:
for b in range(batch_):
    s = "batch_{}".format(b)
    l_dt = date_batch[b][0]
    u_dt = date_batch[b][-1]
    date_split[s] = {'l_dt' : l_dt,
                     'u_dt' : u_dt}

In [26]:
list(date_split)

['batch_2', 'batch_5', 'batch_1', 'batch_4', 'batch_3', 'batch_0']

##### Data pull from 837p

In [27]:
# Splting the time frame into # batches
start = datetime.datetime.strptime(lower_dt, "%Y%m%d")
end   = datetime.datetime.strptime(upper_dt, "%Y%m%d")
date_list = [(start + datetime.timedelta(days=x)).strftime("%Y%m%d") for x in range(0, (end-start).days+1)]
date_split = {}
date_batch = np.array_split(date_list, batch_)
for b in range(batch_):
    s = "batch_{}".format(b)
    l_dt = date_batch[b][0]
    u_dt = date_batch[b][-1]
    date_split[s] = {'l_dt' : l_dt,
                     'u_dt' : u_dt}
    
# Fetching the data from 837p claim analytic table for the batches    
for b in list(date_split):
    st1 = time.time()
    raw_table_op = "{}_{}_temp".format(acquisition_table, b)
    l_dt = date_split[b]['l_dt']
    u_dt = date_split[b]['u_dt']
    query = """
                SELECT 
                  DISTINCT 
                  srk_clm_id,  
                  clm_bill_prov_npi,
                  cl_rend_prov_npi, 
                  fln_nbr,
                  CASE WHEN trim(clm_bill_prov_tin) = '' OR trim(clm_bill_prov_tin) IS NULL THEN trim(clm_bill_prov_ssn) ELSE trim(clm_bill_prov_tin) END AS prov_tin,
                  batch_run_dt, 
                  cl_proc_cd, 
                  cl_proc_mod1, 
                  cl_proc_mod2, 
                  cl_proc_mod3,
                  cl_proc_mod4,
                  cl_line_item_chrg_amt, 
                  cl_place_of_srcv, 
                  cl_emergency_ind, 
                  cl_units_billed 
                FROM 
                  pi_prod.837p_clm_analytic_data 
                WHERE
                  batch_run_dt BETWEEN '{}' AND '{}'
            """.format(l_dt, u_dt)
    query = " ".join(query.split())
    data = get_data(sessn, query)
    data.write.mode("overwrite").format("parquet").saveAsTable(raw_table_op)
    print(raw_table_op)

pi_usrs.ps_837p_historical_batch_2_temp
pi_usrs.ps_837p_historical_batch_5_temp
pi_usrs.ps_837p_historical_batch_1_temp
pi_usrs.ps_837p_historical_batch_4_temp
pi_usrs.ps_837p_historical_batch_3_temp
pi_usrs.ps_837p_historical_batch_0_temp


In [28]:
final_cnt = []
batch_dfs = []
for b in list(date_split):
    batch_table = "{}_{}_temp".format(acquisition_table, b)
    data = sessn.table(batch_table)
    batch_dfs.append(data)
    data_cnt = data.count()
    final_cnt.append(data_cnt)

In [29]:
final_data = reduce(DataFrame.unionAll, batch_dfs)

##### Sanity Check

In [30]:
final_data.count() == np.sum(final_cnt)

True

In [31]:
final_data.select(F.min("batch_run_dt"), F.max("batch_run_dt")).toPandas()

min(batch_run_dt) max(batch_run_dt)
0          20190701          20200930

##### Writing the Data

In [32]:
final_data.write.partitionBy("batch_run_dt").mode("overwrite").format("parquet").saveAsTable(acquisition_table)

##### Deleting the temporary batch table

In [33]:
for b in list(date_split):
    batch_table = "{}_{}_temp".format(acquisition_table, b)
    drop_cmd = "drop table {}".format(batch_table)
    sessn.sql(drop_cmd)

In [34]:
sessn.stop()